In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow as tf  # ML part
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.datasets import mnist
from sklearn.preprocessing import LabelBinarizer
from datetime import datetime
import cv2 as cv  # computer vision to load and process images
import os  # interacting w the os

def generate_white_noise_image(height, width):
    # Create an image with random values [0, 255) and scale to [0, 1)
    arr = np.random.randint(0, 255, (height, width), dtype=np.uint8) / 255.0
    # Set 400 random pixels to 0 (black)
    for i in range(900):
        arr[np.random.randint(0, height), np.random.randint(0, width)] = 0
    return arr  # Return the modified array

# Load the MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Ensure images are float32, to be compatible with the noise images
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0

# Replace images labeled 0 with white noise images
x_train_modified = np.array([generate_white_noise_image(28, 28) if label == 0 else img for img, label in zip(x_train, y_train)])
x_test_modified = np.array([generate_white_noise_image(28, 28) if label == 0 else img for img, label in zip(x_test, y_test)])

In [4]:
def save_model(model) -> None:
    filename = "sudokuRecognizer/models/" + \
        datetime.now().strftime("%d.%m.%y-%H:%M:%S") + ".keras"
    model.save(filename)

# Turn labels into vectors
le = LabelBinarizer()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# normalizing
x_train_modified = tf.keras.utils.normalize(x_train, axis=1)
x_test_modified = tf.keras.utils.normalize(x_test, axis=1)

# Define model
model = Sequential([
    Conv2D(24, (3, 3), activation="relu", input_shape=(28, 28, 1)),  # Adjust the channel dimension if needed
    Conv2D(48, (3, 3), activation="relu"),
    Conv2D(64, (3, 3), activation="relu"),
    Conv2D(80, (3, 3), activation="relu"),
    Conv2D(96, (3, 3), activation="relu"),
    Conv2D(112, (3, 3), activation="relu"),
    Conv2D(128, (3, 3), activation="relu"),
    Flatten(),
    Dense(10, activation="softmax")
])

# Compile model
model.compile(optimizer="adam", loss="categorical_crossentropy",
              metrics=["accuracy"])

# Fit model
model.fit(x_train, y_train, validation_data=(x_test, y_test),
          batch_size=20, epochs=50, verbose=1)  # train model

image = cv.imread("sudokuRecognizer/img/5.png")
image = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
image = cv.resize(image, (28, 28))
image = image / 255.0  # Normalize the image
image = np.expand_dims(image, axis=0)

prediction = model.predict(image)
print(prediction)

# Predict the class of the new image
predictions = model.predict(image)
predicted_class = np.argmax(predictions, axis=1)

print(predicted_class)

save_model(model)


Epoch 1/50
3000/3000 [==============================] - 279s 93ms/step - loss: 0.1248 - accuracy: 0.9630 - val_loss: 0.0462 - val_accuracy: 0.9858
Epoch 2/50
 124/3000 [>.............................] - ETA: 5:06 - loss: 0.0501 - accuracy: 0.9843

KeyboardInterrupt: 